Facciamo delle prove a convertire file e preprocessare!

In [3]:
import os
from pdf2image import convert_from_path

input_file ="../data/raw/full/BerolPhill1516.pdf"
output_path="../data/raw/pages/BerolPhill1516"
os.makedirs(output_path, exist_ok=True)

pages = convert_from_path(
    input_file,
    dpi=200,              
    first_page=13,
    last_page=396
)

for i, page in enumerate(pages, start=1):
    out_path = os.path.join(output_path, f"page_{i:03d}.png")
    page.save(out_path, "PNG")
    print("Saved:", out_path)

Saved: ../data/raw/pages/BerolPhill1516\page_001.png
Saved: ../data/raw/pages/BerolPhill1516\page_002.png
Saved: ../data/raw/pages/BerolPhill1516\page_003.png
Saved: ../data/raw/pages/BerolPhill1516\page_004.png
Saved: ../data/raw/pages/BerolPhill1516\page_005.png
Saved: ../data/raw/pages/BerolPhill1516\page_006.png
Saved: ../data/raw/pages/BerolPhill1516\page_007.png
Saved: ../data/raw/pages/BerolPhill1516\page_008.png
Saved: ../data/raw/pages/BerolPhill1516\page_009.png
Saved: ../data/raw/pages/BerolPhill1516\page_010.png
Saved: ../data/raw/pages/BerolPhill1516\page_011.png
Saved: ../data/raw/pages/BerolPhill1516\page_012.png
Saved: ../data/raw/pages/BerolPhill1516\page_013.png
Saved: ../data/raw/pages/BerolPhill1516\page_014.png
Saved: ../data/raw/pages/BerolPhill1516\page_015.png
Saved: ../data/raw/pages/BerolPhill1516\page_016.png
Saved: ../data/raw/pages/BerolPhill1516\page_017.png
Saved: ../data/raw/pages/BerolPhill1516\page_018.png
Saved: ../data/raw/pages/BerolPhill1516\page_0

In [15]:
import cv2
import os
from tqdm import tqdm


input_dir = "../data/raw/pages/BerolPhill1516"
gsc_dir = "../data/processed/gsc"
bzd_dir = "../data/processed/bzd"
ctr_dir = "../data/processed/ctr"

os.makedirs(gsc_dir,exist_ok=True)
os.makedirs(bzd_dir,exist_ok=True)
os.makedirs(ctr_dir,exist_ok=True)

images = sorted([f for f in os.listdir(input_dir) if f.endswith(".png")])

print(f"We will convert {len(images)} pages!!")

for name in tqdm(images, desc="STO LAVORANDO"):
    path = os.path.join(input_dir, name)

    gsc_path = os.path.join(gsc_dir, name)
    bzd_path = os.path.join(bzd_dir, name)
    ctr_path = os.path.join(ctr_dir, name)

    gsc_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    #save gsc
    cv2.imwrite(gsc_path, gsc_img)

    _, bzd_img = cv2.threshold(
        gsc_img, 
        0,
        255,
        cv2.THRESH_BINARY + cv2.THRESH_OTSU
    )

    #save bzd
    cv2.imwrite(bzd_path, bzd_img)

    blurred = cv2.GaussianBlur(gsc_img, (3,3), 0)
    edges = cv2.Canny(blurred, threshold1=50, threshold2=150)
    edges_inv = cv2.bitwise_not(edges)

    #sace ctr
    cv2.imwrite(ctr_path, edges_inv)

We will convert 384 pages!!


STO LAVORANDO: 100%|██████████| 384/384 [01:36<00:00,  3.99it/s]
